<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from db import FireBase

In [2]:
import pandas as pd
import json
import os

In [3]:
os.listdir('./data')

['analyst.pkl',
 'company.pkl',
 'penny_performance.pkl',
 'performance.pkl',
 'performance_combined.pkl',
 'performance_penny_combined.pkl',
 'prices.pkl']

In [15]:
df_performance = pd.read_pickle('./data/performance.pkl')

In [16]:
df_performance = df_performance.set_index('date', drop=True)
df_performance.head()

,sym,RevenueTTM,TotalCash,ExDividendDate,Dividend,DividendYield,PriceToBook,PriceToSales,EnterpriseValue,PriceToCashFlow,PE,ReturnonEquity,FreeCashFlow,TotalDebtToEquity,PricePerformance52Weeks,PricePerformanceYTD,TotalReturn1Yr,TotalReturn3Yr,VolumeAvg90Day,price
date,,,,,,,,,,,,,,,,,,,,
2019-08-09,A,5019.0,2155.0,0,0.164,0.94402,4.29809,4.40182,2.167070e+10,16.57368,19.8,23.1598,887.0,37.29007,3.13149,3.00919,4.04369,14.79356,2.59274,68.64
2019-08-12,A,5019.0,2155.0,0,0.164,0.95571,4.24551,4.34797,2.140126e+10,16.37095,19.2,23.1598,887.0,37.29007,3.59191,1.74918,4.50818,14.09856,2.58484,66.78
2019-08-13,A,5019.0,2155.0,0,0.164,0.98454,4.12119,4.22065,2.076411e+10,15.89156,19.5,23.1598,887.0,37.29007,0.55841,-1.23036,1.44784,12.47880,2.60485,67.64
2019-08-14,A,5019.0,2155.0,0,0.164,0.96984,4.18366,4.28463,2.108427e+10,16.13245,18.9,23.1598,887.0,37.29007,2.57810,0.26682,3.48540,13.04428,2.61616,65.59
2019-08-15,A,5090.0,1765.0,0,0.164,1.00015,4.28332,4.06782,2.036590e+10,16.05056,20.8,22.0179,918.0,36.93453,-1.73783,-2.77201,-0.86870,11.89052,2.63078,69.59


In [20]:
df_performance.columns

Index(['sym', 'RevenueTTM', 'TotalCash', 'ExDividendDate', 'Dividend',
       'DividendYield', 'PriceToBook', 'PriceToSales', 'EnterpriseValue',
       'PriceToCashFlow', 'PE', 'ReturnonEquity', 'FreeCashFlow',
       'TotalDebtToEquity', 'PricePerformance52Weeks', 'PricePerformanceYTD',
       'TotalReturn1Yr', 'TotalReturn3Yr', 'VolumeAvg90Day', 'price'],
      dtype='object')

In [17]:
fb = FireBase()

In [21]:
post = {}
all_symbols = df_performance['sym'].unique()
sym_df = df_performance.groupby('sym')

x = 0
for sym in all_symbols:
    x += 1
    post[sym] = sym_df.get_group(sym).drop('sym', axis=1).reset_index(drop=True).to_dict()
    if x == 100:
        print('posting ->', sym, end='\r')
        req = fb.refresh_all_data(post)
        print('posted ->', sym, end='\r')
        post = {}
        x = 0

req = fb.refresh_all_data(post)

In [22]:
date_range = df_performance.query("sym == 'AAPL'").reset_index().date.map(lambda x: x.strftime("%m-%d-%Y")).to_dict()
df_performance.reset_index(drop=True, inplace=True)
column_index = list(df_performance.columns)
column_index.remove('sym')
column_index = dict(zip(range(len(column_index)), column_index))
all_symbols = df_performance['sym'].unique()

In [23]:
index

NameError: name 'index' is not defined

In [ ]:
symbol_index = tuple(all_symbols)
symbol_index

In [ ]:
index = {
    'column': column_index,
    'date_range': date_range,
    'stocks': symbol_index
}
req = fb.refresh_index(index)

In [ ]:
import time

```python
# Test

aal = {}
sym_df = df_performance.groupby('sym')
aal = sym_df.get_group('AAL').drop('sym', axis=1).reset_index(drop=True).to_dict()
aal['AAL/Argus/322'] = 7
req = fb.refresh_all_data(aal)
```

> As this tests works, if we want to update a date i.e adding 322 we will have to generate the url for each column/key


In [ ]:
df_performance.to_json()

In [9]:
# Refresh

post={}
aal = sym_df.get_group('AAL').drop('sym', axis=1).reset_index(drop=True).to_json()
post['AAL'] = aal
req = fb.refresh_all_data(post)

In [8]:
index = fb.get_index()['data']
dr = index['date_range']

In [74]:
index.keys()

dict_keys(['column', 'date_range'])

In [10]:
idx = pd.to_datetime(dr)

In [75]:
def get_df_from_col_sym(col, sym):
    if isinstance(col, str):
        return pd.DataFrame(fb.get_sym_col(sym, col)['data'], index=idx, columns=[col])
    elif isinstance(col, list):
        return pd.concat([pd.DataFrame(fb.get_sym_col(sym, c)['data'],
                                       index=idx, columns=[c])
                          for c in col], axis=1)

def get_graph(col, sym):
    df = get_df_from_col_sym(col, sym)
    fig = px.line(df, y=col)
    del df
    return fig

In [78]:
def get_cs(col, sym):
    print(col)

In [82]:
list(tuple('price'))

['p', 'r', 'i', 'c', 'e']

In [76]:
import plotly.express as px

In [77]:
fig = get_graph(['price', 'RevenueTTM'], 'AAPL')
fig.show()

In [65]:
post = {}
all_symbols = df_performance['sym'].unique()
sym_df = df_performance.groupby('sym')
x = 0
for sym in all_symbols:
    x += 1
    post[sym] = sym_df.get_group(sym).drop('sym', axis=1).reset_index(drop=True).to_dict()
    if x == 100:
        print('posting ->', sym)
        req = fb.refresh_all_data(post)
        post = {}
        x = 0
print('posting ->', sym)
req = fb.refresh_all_data(post)

posting -> AMKR
posting -> AVP
posting -> BSET
posting -> CINR
posting -> CSTM
posting -> DRNE
posting -> ETR
posting -> FRHC
posting -> GWPH
posting -> IDN
posting -> KAR
posting -> LSCC
posting -> MODN
posting -> NHTC
posting -> ONVO
posting -> PLOW
posting -> REFR
posting -> SFLY
posting -> SSTK
posting -> TIF
posting -> UFS
posting -> WBS
